In [65]:
import os

import numpy as np

import torch
import torch.optim as optim
import torch.utils.data.dataset as dataset
import torch.utils.data.dataloader as dataloader
import torchvision

from typing import Any

from tqdm import tqdm
import matplotlib.pyplot as plt

In [66]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
2


In [67]:
class BaseDataset(dataset.Dataset):
    def __init__(self, data: torch.Tensor, label: torch.Tensor, transform=None) -> None:
        super().__init__()
        self.data = data
        self.label = label
        self.transform = transform
    
    def __getitem__(self, index) -> Any:
        x = self.data[index]
        y = torch.zeros((10,)).int()
        y[self.label[index]] = 1
        if self.transform:
            x = self.transform(x)
        return x, y
    
    def __len__(self):
        return self.data.size(0)

def dataset_split(data: torch.Tensor, label: torch.Tensor, split_beg: float, split_end: float, transform=None) -> dataset.Dataset:
    assert split_beg >= 0.0 and split_end <= 1.0, f"invalid split range: {split_beg}-{split_end}"
    assert data.size(0) == label.size(0), f"data, label unmatched: {data.size(0)}, {label.size(0)}"
    
    num_sample = data.size(0)
    data_split = data[int(num_sample*split_beg):int(num_sample*split_end)]
    label_split = label[int(num_sample*split_beg):int(num_sample*split_end)]
    
    return BaseDataset(data_split, label_split, transform)

In [68]:
mnist_dataset = torchvision.datasets.MNIST(root="data", download=False)
print(mnist_dataset)

def normalize_01(data: torch.Tensor):
    assert len(data.shape) > 1 # [batch, [sample_feat, ...]]
    
    batch_transed = []
    for sample in data:
        to_float = sample.float()
        the_min = to_float.min()
        the_max = to_float.max()
        batch_transed.append((to_float - the_min) / (the_max - the_min + 1e-5))
    
    return torch.stack(batch_transed)

train_X = normalize_01(mnist_dataset.train_data)
train_Y = mnist_dataset.train_labels
eval_X = train_X[:int(len(train_X)*0.1)]
eval_Y = train_Y[:int(len(train_Y)*0.1)]
train_X = train_X[len(eval_X):]
train_Y = train_Y[len(eval_Y):]
test_X = normalize_01(mnist_dataset.test_data)
test_Y = mnist_dataset.test_labels

ratio_origin = 0.3
ratio_pesudo = 1.0 - ratio_origin

train_X_origin = train_X[:int(len(train_X)*ratio_origin) ]
train_Y_origin = train_Y[:int(len(train_Y)*ratio_origin) ]
train_X_pesudo = train_X[ int(len(train_X)*ratio_origin):]
train_Y_pesudo = train_Y[ int(len(train_Y)*ratio_origin):]

print(train_X_origin.shape)
print(train_Y_origin.shape)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
torch.Size([16200, 28, 28])
torch.Size([16200])


In [69]:
# https://pytorch-tutorial.readthedocs.io/en/latest/tutorial/chapter03_intermediate/3_2_1_cnn_convnet_mnist/
import torch.nn as nn
import torch.nn.functional as F
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # 28x28
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(20*10*10, 500)
        self.fc2 = nn.Linear(500, 10)
    
    def forward(self, x: torch.Tensor):
        in_size = x.size(0)
        out = self.conv1(x) # 24
        out = F.relu(out)
        
        out = F.max_pool2d(out, 2, 2) # 12
        
        out = self.conv2(out)
        out = F.relu(out)
        
        out = out.view(in_size, -1)
        
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out, dim=1) # perform softmax on sample dim
        
        return out

In [70]:
def train(model: nn.Module, optimizer: optim.Optimizer, scheduler: optim.lr_scheduler._LRScheduler, device: torch.device, dataset: dataset.Dataset, batch_size: int, epoch: int):
    data_loader = dataloader.DataLoader(dataset, batch_size, shuffle=True)
    
    model.train()
    for epoch_idx in range(epoch):
        for batch_idx, (data, label) in tqdm(enumerate(data_loader), desc=f"epoch {epoch_idx + 1}", ncols=100):
            data, label = data.to(device), label.to(device) # move data from host to device
            optimizer.zero_grad()
            output = model(data)
            loss = F.cross_entropy(output, label)
            loss.backward()
            optimizer.step()
            if (batch_idx + 1) % 100 == 0:
                tqdm.write(f"train epoch {epoch_idx + 1}/{epoch} loss: {loss.mean().item()}")
        scheduler.step()

def test(model: nn.Module, device: torch.device, dataset: dataset.Dataset, batch_size: int):
    data_loader = dataloader.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    test_loss_sum = 0.0
    model.eval()
    with torch.no_grad():
        for data, label in data_loader:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss_sum += F.cross_entropy(output, label, reduction="sum").item() # sum up batch loss
            

In [71]:
rt_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNet().to(torch.device("cuda"))
optimizer = optim.Adam(params=model.parameters(), lr=0.1)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

train(model, optimizer, scheduler, rt_device, dataset_split(train_X_origin, train_Y_origin, 0.0, 1.0), 10, 10)


epoch 1: 0it [00:00, ?it/s]


RuntimeError: Given groups=1, weight of size [10, 1, 5, 5], expected input[1, 10, 28, 28] to have 1 channels, but got 10 channels instead

In [1]:
import torch

torch.nn.functional.one_hot(torch.tensor([1,2,3,4,5]), 10)

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [3]:
pred = torch.tensor([[1,2],[3,4]])
gdth = torch.tensor([[1,3],[2,4]])

accu = pred.eq(gdth).int().sum()
print(accu.item())

2
